In [1]:
!pip install transformers
!pip install pandas
!pip install hazm
!pip install nltk
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 50.4 MB/s 
     |████████████████████████████████| 101 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 57.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 5.0 MB/s 
     |████████████████████████████████| 1.4 MB 62.5 MB/s 
     |████████████████████████████████| 233 kB 69.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=0a8b61ad0eb908a92bae6c8034fcb6d029899cc4f2520bc06ef020ec8cf84e69
  Sto

In [2]:
import pandas as pd
from hazm import *
import tqdm
import json
import codecs
import numpy as np
from transformers import BigBirdModel, AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
import random
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
MODEL_NAME = "SajjadAyoubi/distil-bigbird-fa-zwnj"
model = BigBirdModel.from_pretrained(MODEL_NAME, block_size=32)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/837 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300M [00:00<?, ?B/s]

Some weights of the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj were not used when initializing BigBirdModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdModel were not initialized from the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj and are newly initialized: ['bert.pooler.

Downloading:   0%|          | 0.00/365 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
import random
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    if is_tf_available():
        tf.random.set_seed(seed)

set_seed(1)

In [5]:
news_dict = {}
with open('/content/news.json', "r", encoding="utf-8") as text_file:
        news_dict = json.loads(text_file.read())

df = pd.DataFrame.from_dict(news_dict, orient='index')
df

,title,subject,content
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی:اصلاحیه قانون به ...
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین،فدا حسین مالکی در مورد ...
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...


In [6]:
normalizer = Normalizer()

normalized_news_dict = {}
for idx,value in tqdm.tqdm(news_dict.items()):
    normalized_news_dict[idx] = {"title" : value['title'] ,"subject" : value["subject"], "content":normalizer.normalize(value['content'])}
  
news_df = pd.DataFrame.from_dict(normalized_news_dict, orient='index')

100%|██████████| 14876/14876 [00:20<00:00, 714.12it/s] 


In [7]:
new_stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('/content/new_stopwords.txt','r','utf-8').readlines()]
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('/content/stopwords.txt','r','utf-8').readlines()]

In [8]:
tokenized_words = [word_tokenize(_) for _ in news_df.content]

In [9]:
removed_tokenized_words = []
for per_content_words in tokenized_words:
  words = []
  for word in per_content_words:
    if word not in stopwords + new_stopwords:
      words.append(word)
  removed_tokenized_words.append(words)

news_df['word_tokenize'] = removed_tokenized_words
news_df

,title,subject,content,word_tokenize
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"[گزارش, همشهری, آنلاین, روزنامه, فرهیختگان, شم..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"[همشهری‌آنلاین-, امیرمحمدحسینی, اصلاحیه, قانون..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"[گزارش, همشهری‌آنلاین, فدا, حسین, مالکی, قطعنا..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"[ЩҮЩ…ШҙЩҮШұЫҢ, ШўЩҶЩ„Ш§ЫҢЩҶ, Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"[گزارش, همشهری‌آنلاین, آیت‌الله, سید, ابراهیم,..."
...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"[گزارش, همشهری, آنلاین, صعود, پرتغال, لهستان, ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"[گزارش, همشهری, آنلاین, تیم, ملی, مصر, مصاف, س..."


In [10]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt == 1:
        return stemmer.stem(tok)
    if opt == 2:
        return lemmatizer.lemmatize(tok)

In [11]:
opt = 2
lemmatize_stemming_words = []
for per_content_words in news_df.word_tokenize:
  words = []
  for word in per_content_words:
    words.append(get_lemma_set(word,opt))
  lemmatize_stemming_words.append(words)
news_df['word_lemmatize'] = lemmatize_stemming_words
news_df

,title,subject,content,word_tokenize,word_lemmatize
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"[گزارش, همشهری, آنلاین, روزنامه, فرهیختگان, شم...","[گزارش, همشهری, آنلاین, روزنامه, فرهیختگان, شم..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"[همشهری‌آنلاین-, امیرمحمدحسینی, اصلاحیه, قانون...","[همشهری‌آنلاین-, امیرمحمدحسینی, اصلاحیه, قانون..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"[گزارش, همشهری‌آنلاین, فدا, حسین, مالکی, قطعنا...","[گزارش, همشهری‌آنلاین, فدا, حسین, مالکی, قطعنا..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"[ЩҮЩ…ШҙЩҮШұЫҢ, ШўЩҶЩ„Ш§ЫҢЩҶ, Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,...","[ЩҮЩ…ШҙЩҮШұЫҢ, ШўЩҶЩ„Ш§ЫҢЩҶ, Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"[گزارش, همشهری‌آنلاین, آیت‌الله, سید, ابراهیم,...","[گزارش, همشهری‌آنلاین, آیت‌الله, سید, ابراهیم,..."
...,...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,...","[گزارش, همشهری, آنلاین, مسابقات, مقدمات, جام, ..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"[گزارش, همشهری, آنلاین, صعود, پرتغال, لهستان, ...","[گزارش, همشهری, آنلاین, صعود, پرتغال, لهستان, ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,...","[گزارش, همشهری, آنلاین, مسابقات, مقدمات, جام, ..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"[گزارش, همشهری, آنلاین, تیم, ملی, مصر, مصاف, س...","[گزارش, همشهری, آنلاین, تیم, ملی, مصر, مصاف, س..."


In [12]:
words = []
for row_words in news_df['word_lemmatize']:
  words.append(' '.join(row_words))
news_df['clean_text'] = words
news_df = news_df.drop(['word_tokenize','word_lemmatize'],axis=1)
news_df

,title,subject,content,clean_text
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,گزارش همشهری آنلاین روزنامه فرهیختگان شماره ام...
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,همشهری‌آنلاین- امیرمحمدحسینی اصلاحیه قانون کار...
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,گزارش همشهری‌آنلاین فدا حسین مالکی قطعنامه ضد ...
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ ШӯШ...
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,گزارش همشهری‌آنلاین آیت‌الله سید ابراهیم رییس ...
...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,گزارش همشهری آنلاین مسابقات مقدمات جام جهانی ۲...
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,گزارش همشهری آنلاین صعود پرتغال لهستان ۵ تیم آ...
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,گزارش همشهری آنلاین مسابقات مقدمات جام جهانی ۲...
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,گزارش همشهری آنلاین تیم ملی مصر مصاف سنگال تسا...


In [13]:
target_category = news_df['subject'].unique()
target_category = target_category.tolist()
news_df['subject_id'] = news_df['subject'].factorize()[0]

In [19]:
class NewsDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor([self.labels[idx]])
    return item
  def __len__(self):
    return len(self.labels)

def create_x_y_classification(data, df):
    x = []
    for k, v in data.items():
      x.append(v['clean_text'])
    y = np.array(df.subject_id.values)
    return x, y

def split_data_train_test_val(x, y, test_size=0.3, val_size=0.5, random_state=0, shuffle=True):
    x_train_tr, x_testval_tr, y_train_tr, y_testval_tr = train_test_split(x, y, test_size=test_size, random_state=random_state, shuffle=shuffle)
    x_val_tr, x_test_tr, y_val_tr, y_test_tr = train_test_split(x_testval_tr, y_testval_tr, test_size=val_size, random_state=random_state, shuffle=shuffle)
    return x_train_tr, x_val_tr, x_test_tr, y_train_tr, y_val_tr, y_test_tr

def encoding(x_train_tr, x_val_tr, x_test_tr):
    train_encodings = tokenizer(x_train_tr, truncation=True, padding=True)
    val_encodings = tokenizer(x_val_tr, truncation=True, padding=True)
    test_encodings = tokenizer(x_test_tr, truncation=True, padding=True)
    return train_encodings, val_encodings, test_encodings

def create_encoded_dataset(train_encodings, val_encodings, test_encodings):
    train_dataset = NewsDataset(train_encodings, y_train_tr)
    val_dataset = NewsDataset(val_encodings, y_val_tr)
    test_dataset = NewsDataset(test_encodings, y_test_tr)
    return train_dataset, val_dataset, test_encodings

def create_tran_model(model_name=MODEL_NAME, num_labels=11):
    tran_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels).to("cuda")
    return tran_model

def train_transformer_classification(tran_model, train_dataset, val_dataset, epoch=3, batch_size=16):
    training_args = TrainingArguments(
      output_dir="./results",
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epoch,
      weight_decay=0.01,
    )

    trainer = Trainer(
        model=tran_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()
    return trainer

def save_transformer_trainer(trainer):
    trainer.save_model('/content/models/classification_transformers')

def load_transformer():
  return AutoModelForSequenceClassification.from_pretrained('/content/models/classification_transformers', local_files_only=True).to("cuda")

def get_prediction(text, tran_model, tokenizer=tokenizer):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
    outputs = tran_model(**inputs)
    probs = outputs[0].softmax(1)
    return probs.argmax().item()

def predict(x_test, tran_model):
    y_pred = []
    for idx, x in enumerate(x_test):
      y_pred.append(get_prediction(x, tran_model))
    return y_pred

def visualize_clf_transformer_csv(x, y, y_preds, categories):
    test_comparision = []
    for idx, ix in enumerate(x):
      pred_res = categories[y_preds[idx]]
      true_res = categories[y[idx]]
      test_comparision.append([true_res, pred_res])

    import csv
    with open('predicted_transformers.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["true_label", "predicted_label"])
        for row in test_comparision:
          writer.writerow(row)

def evaluate_transformer_clf(y, y_preds):
    confusion_matrix_evaluate = confusion_matrix(y, y_preds)
    accuracy_score_evaluate = accuracy_score(y, y_preds) * 100
    f1_score_evaluate = f1_score(y, y_preds, average='macro')

    print(f"accuracy score: {accuracy_score_evaluate}")
    print('-----------------------------------------------------------------------')
    print(f"f1 score: {f1_score_evaluate}")
    print('-----------------------------------------------------------------------')
    print(f"confusion matrix:\n {confusion_matrix_evaluate}")

    plt.matshow(confusion_matrix_evaluate)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


In [ ]:
pre_processed_news_dict = news_df.apply(lambda x:x.to_dict(), axis=1)
x, y = create_x_y_classification(pre_processed_news_dict, news_df)
x_train_tr, x_val_tr, x_test_tr, y_train_tr, y_val_tr, y_test_tr = split_data_train_test_val(x, y)
train_encodings, val_encodings, test_encodings = encoding(x_train_tr, x_val_tr, x_test_tr)
train_dataset, val_dataset, test_encodings = create_encoded_dataset(train_encodings, val_encodings, test_encodings)
tran_model = create_tran_model()
trainer = train_transformer_classification(tran_model, train_dataset, val_dataset)
save_transformer_trainer(trainer)
tran_model_loaded = load_transformer()
y_pred_tr = predict(x_test_tr, tran_model_loaded)
evaluate_transformer_clf(y_test_tr, y_pred_tr)


In [ ]:
!zip -r /content/models.zip /content/models
from google.colab import files
files.download("/content/models.zip")